# Modulos

In [ ]:
import os
from pathlib import Path
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import sys  # Acceso a variables y funciones del sistema
import re  # Expresiones regulares para procesamiento de texto
import os  # Operaciones del sistema de archivos


sys.path.append(os.path.abspath("c:/Users/osmarrincon/Documents/capresoca-data-automation"))
# Importar función y clase personalizada del proyecto
from src.file_loader import cargar_maestros_ADRES  # Función para cargar archivos maestros ADRES
from src.data_cleaning import BduaReportProcessor      # Clase para limpiar y normalizar población Maestro ADRES
from src.data_cleaning import DataCleaner # Clase para limpiar y normalizar DataFrames de Pandas

# Rutas

In [ ]:
Raiz = r"\\Servernas\AYC2\(Z)RSERVER(Z)\LORENA CARDOZO\RED DE SERVICIOS"

Hoja_Com = "COMPENSADOS"
Hoja_LMA = "BD"
R_Compensados= Raiz + r"\COMPENSADOS\COMPENSADOS.xlsx"
r_LMA  = Raiz + r"\LMA\LMA DICIEMBRE v1.xlsx"
r_MsEPS025 = Raiz + r"\MS\EPS025MS0029012026.TXT"
r_MsEPSC25 = Raiz + r"\MS\EPSC25MC0029012026.TXT"

Hoja_pnEPSC25 = "CONTRIBUTIVO"
r_P_Nacional_EPSC25 = Raiz + r"\PORTABILIDAD\PORTABILIDAD_NACIONAL_REGIMEN_CONTRIBUTIVO_DICIEMBRE25.xlsx"
Hoja_pnEPSS025 = "SUBSIDIADO"
r_P_Nacional_EPS025 = Raiz + r"\PORTABILIDAD\PORTABILIDAD_NACIONAL_REGIMEN_SUBSIDIADO_DICIEMBRE25.xlsx"

r_P_Regional_EPSC25 = Raiz + r"\PORTABILIDAD\PORTABILIDAD_REGIONAL_REGIMEN_CONTRIBUTIVO_DICIEMBRE25.xlsx"
r_P_Regional_EPS025 = Raiz + r"\PORTABILIDAD\PORTABILIDAD_REGIONAL_REGIMEN_SUBSIDIADO_DICIEMBRE25.xlsx"

r_Municipios_SIE = r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Capresoca\AlmostClear\SIE\codificación de variables categóricas\Reporte_MUNICIPIOS_2025_05_14.csv"



# Dataframes

In [ ]:
df_municipios_sie = pd.read_csv(r_Municipios_SIE, encoding='ansi', sep=';')

In [ ]:
df_compensados = pd.read_excel(R_Compensados, sheet_name=Hoja_Com, header=0, dtype=str)
df_lma = pd.read_excel(r_LMA, sheet_name=Hoja_LMA, header=0, dtype=str)

In [ ]:
# Cargar portabilidad nacional contributivo
df_PN_contributivo = pd.read_excel(r_P_Nacional_EPSC25, sheet_name=Hoja_pnEPSC25, header=0, dtype=str)
df_PN_contributivo['regimen_pn'] = 'CONTRIBUTIVO'

# Cargar portabilidad nacional subsidiado
df_PN_subsidiado = pd.read_excel(r_P_Nacional_EPS025, sheet_name=Hoja_pnEPSS025, header=0, dtype=str)
df_PN_subsidiado['regimen_pn'] = 'SUBSIDIADO'

# Unificar en un solo dataframe
df_PN = pd.concat([df_PN_contributivo, df_PN_subsidiado], ignore_index=True)

print(f"Registros portabilidad contributivo: {len(df_PN_contributivo)}")
print(f"Registros portabilidad subsidiado: {len(df_PN_subsidiado)}")
print(f"Total portabilidad nacional: {len(df_PN)}")
print(f"\nPrimeros registros:")
print(df_PN.head())

In [ ]:
# Cargar portabilidad nacional contributivo
df_PR_contributivo = pd.read_excel(r_P_Regional_EPSC25, sheet_name=Hoja_pnEPSC25, header=0, dtype=str)
df_PR_contributivo['regimen_pn'] = 'CONTRIBUTIVO'

# Cargar portabilidad nacional subsidiado
df_PR_subsidiado = pd.read_excel(r_P_Regional_EPS025, sheet_name=Hoja_pnEPSS025, header=0, dtype=str)
df_PR_subsidiado['regimen_pn'] = 'SUBSIDIADO'

# Unificar en un solo dataframe
df_PR = pd.concat([df_PR_contributivo, df_PR_subsidiado], ignore_index=True)
print(f"Registros portabilidad contributivo: {len(df_PR_contributivo)}")
print(f"Registros portabilidad subsidiado: {len(df_PR_subsidiado)}")
print(f"Total portabilidad nacional: {len(df_PR)}")
print(f"\nPrimeros registros:")
print(df_PR.head())

In [ ]:
maestro_ADRES = cargar_maestros_ADRES(r_MsEPS025, r_MsEPSC25)

# Depurar dataframes

In [ ]:
df_compensados = df_compensados.drop(columns=['TOTAL', 'Nuevo_Nombre2', 'Nuevo_Nombre1', 'Tipo_Afiliado', 'Estado_Registro', 'Serial_BDUA', 'Recursos_Fondo_PyP', 'Upc_Reconocer', 'Total_Dias_Cotizados', 'Total_Dias_Compensados', 'Periodo_compensado', 'Fecha_Proceso_Giro_Compensación', 'Código_EPS'])
df_lma = df_lma[['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 12', 'Unnamed: 13']]

In [ ]:
df_lma = df_lma.rename(columns={
    'Unnamed: 1': 'Tp_Dc',
    'Unnamed: 2': 'Num_coti',
    'Unnamed: 12': 'Departamento',
    'Unnamed: 13': 'Municipio'
})

In [ ]:
df_compensados['compensados'] = 'COMPENSADOS'
df_lma['compensados'] = 'LMA'

In [ ]:
df_lma = df_lma.dropna(subset=['Tp_Dc'])

In [ ]:
df_unificado = pd.concat([df_compensados[['Tp_Dc', 'Num_coti', 'Departamento', 'Municipio', 'compensados']], df_lma], ignore_index=True)

In [ ]:
df_unificado = df_unificado.merge(
    maestro_ADRES[['TPS_IDN_ID', 'HST_IDN_NUMERO_IDENTIFICACION', 'ENT_ID']],
    left_on=['Tp_Dc', 'Num_coti'],
    right_on=['TPS_IDN_ID', 'HST_IDN_NUMERO_IDENTIFICACION'],
    how='left'
).drop(columns=['TPS_IDN_ID', 'HST_IDN_NUMERO_IDENTIFICACION'])

In [ ]:
# Validar duplicados en df_unificado
duplicados = df_unificado[df_unificado.duplicated(subset=['Tp_Dc', 'Num_coti'], keep=False)]
cantidad_duplicados = duplicados.duplicated(subset=['Tp_Dc', 'Num_coti']).sum()

print(f"Cantidad de registros duplicados: {cantidad_duplicados}")
print(f"Total de filas con duplicados: {len(duplicados)}")

# Obtener los primeros 3 pares duplicados únicos
pares_duplicados = duplicados[['Tp_Dc', 'Num_coti']].drop_duplicates().head(3)

print(f"\nPrimeros 3 ejemplos de registros duplicados:\n")
for idx, (tp_dc, num_coti) in enumerate(pares_duplicados.values, 1):
    print(f"--- Ejemplo {idx}: {tp_dc} - {num_coti} ---")
    registros = duplicados[(duplicados['Tp_Dc'] == tp_dc) & (duplicados['Num_coti'] == num_coti)]
    print(registros)
    print()

In [ ]:
# Validar duplicados en df_PN por tipo_identificacion y numero_identificacion
duplicados_PN = df_PN[df_PN.duplicated(subset=['tipo_identificacion', 'numero_identificacion'], keep=False)]

cantidad_duplicados_PN = duplicados_PN.duplicated(subset=['tipo_identificacion', 'numero_identificacion']).sum()

print(f"Cantidad de registros duplicados: {cantidad_duplicados_PN}")
print(f"Total de filas con duplicados: {len(duplicados_PN)}")

# Obtener los primeros 3 pares duplicados únicos
pares_duplicados_PN = duplicados_PN[['tipo_identificacion', 'numero_identificacion']].drop_duplicates().head(3)

print(f"\nPrimeros 3 ejemplos de registros duplicados:\n")
for idx, (tipo_id, num_id) in enumerate(pares_duplicados_PN.values, 1):
    print(f"--- Ejemplo {idx}: {tipo_id} - {num_id} ---")
    registros = duplicados_PN[(duplicados_PN['tipo_identificacion'] == tipo_id) & (duplicados_PN['numero_identificacion'] == num_id)]
    print(registros[['tipo_identificacion', 'numero_identificacion', 'primer_nombre', 'primer_apellido', 'regimen_pn', 'estado_movilidad']])
    print()

In [ ]:
# Definir la prioridad: 1 es mayor prioridad, 2 es menor
# Contributivo (EPSC25) + COMPENSADOS: máxima prioridad
# Subsidiado (EPS025) + LMA: segunda prioridad
df_unificado['prioridad'] = 4  # default baja prioridad

# Prioridad 1: Contributivo (EPSC25) con COMPENSADOS
df_unificado.loc[
    (df_unificado['ENT_ID'] == 'EPSC25') & 
    (df_unificado['compensados'] == 'COMPENSADOS'), 
    'prioridad'
] = 1

# Prioridad 2: Subsidiado (EPS025) con LMA
df_unificado.loc[
    (df_unificado['ENT_ID'] == 'EPS025') & 
    (df_unificado['compensados'] == 'LMA'), 
    'prioridad'
] = 2

# Prioridad 3: Otros casos
df_unificado.loc[
    (df_unificado['ENT_ID'].isin(['EPSC25', 'EPS025'])) & 
    (df_unificado['prioridad'] == 4), 
    'prioridad'
] = 3

# Ordenar por ID y prioridad, luego eliminar duplicados manteniendo el de mayor prioridad
df_unificado = df_unificado.sort_values(
    by=['Tp_Dc', 'Num_coti', 'prioridad']
).drop_duplicates(
    subset=['Tp_Dc', 'Num_coti'], 
    keep='first'
).drop(columns=['prioridad'])

# Validar resultados
print(f"Registros después de eliminar duplicados: {len(df_unificado)}")

# Verificar que no hay duplicados
duplicados_validacion = df_unificado[df_unificado.duplicated(subset=['Tp_Dc', 'Num_coti'], keep=False)]
print(f"Duplicados restantes: {len(duplicados_validacion)}")

# Mostrar algunos ejemplos de lo que quedó
print("\nEjemplos de registros mantenidos:")
print(df_unificado[['Tp_Dc', 'Num_coti', 'compensados', 'ENT_ID']].head(10))

In [ ]:
# Garantizar formato de dos dígitos para Departamento y tres dígitos para Municipio
df_unificado['Departamento'] = df_unificado['Departamento'].fillna('00').astype(str).str.zfill(2)
df_unificado['Municipio'] = df_unificado['Municipio'].fillna('000').astype(str).str.zfill(3)

print("Formatos aplicados:")
print(f"Departamento - Ejemplos: {df_unificado['Departamento'].unique()[:5]}")
print(f"Municipio - Ejemplos: {df_unificado['Municipio'].unique()[:5]}")
print(f"\nDepartamentos con formato incorrecto: {(df_unificado['Departamento'].str.len() != 2).sum()}")
print(f"Municipios con formato incorrecto: {(df_unificado['Municipio'].str.len() != 3).sum()}")

In [ ]:
# Merge df_unificado con df_PN usando tipo_identificacion y numero_identificacion
df_unificado = df_unificado.merge(
    df_PN[['tipo_identificacion', 'numero_identificacion', 'municipio_receptor', 'departamento_receptor']],
    left_on=['Tp_Dc', 'Num_coti'],
    right_on=['tipo_identificacion', 'numero_identificacion'],
    how='left'
).drop(columns=['tipo_identificacion', 'numero_identificacion'])

# Renombrar columnas de PN
df_unificado = df_unificado.rename(columns={
    'municipio_receptor': 'municipio_receptor_PN',
    'departamento_receptor': 'departamento_receptor_PN'
})

print(f"Registros con Portabilidad Nacional (PN): {df_unificado['municipio_receptor_PN'].notna().sum()}")

# Merge df_unificado con df_PR usando tipo_identificacion y numero_identificacion
df_PR_merge = df_PR[['tipo_identificacion', 'numero_identificacion', 'municipio_receptor', 'departamento_receptor']].copy()
df_PR_merge.columns = ['tipo_identificacion', 'numero_identificacion', 'municipio_receptor_PR', 'departamento_receptor_PR']

df_unificado = df_unificado.merge(
    df_PR_merge,
    left_on=['Tp_Dc', 'Num_coti'],
    right_on=['tipo_identificacion', 'numero_identificacion'],
    how='left'
).drop(columns=['tipo_identificacion', 'numero_identificacion'])

print(f"Registros con Portabilidad Regional (PR): {df_unificado['municipio_receptor_PR'].notna().sum()}")

# Contar registros que tienen ambas (PN y PR)
ambas = (df_unificado['municipio_receptor_PN'].notna()) & (df_unificado['municipio_receptor_PR'].notna())
cantidad_ambas = ambas.sum()

print(f"Registros que tienen AMBAS (PN y PR): {cantidad_ambas}")
print(f"  → Estos serán sobrescritos por PR")

# Crear columnas finales: PR sobrescriba a PN si existe
# Primero usar PR, si no existe usar PN
df_unificado['municipio_receptor'] = df_unificado['municipio_receptor_PR'].fillna(df_unificado['municipio_receptor_PN'])
df_unificado['departamento_receptor'] = df_unificado['departamento_receptor_PR'].fillna(df_unificado['departamento_receptor_PN'])

# Crear columna Tipo_Portabilidad basada en lo que quedó final
df_unificado['Tipo_Portabilidad'] = 'Sin portabilidad'

# Regional: si tiene PR (independientemente de si tiene PN)
df_unificado.loc[df_unificado['municipio_receptor_PR'].notna(), 'Tipo_Portabilidad'] = 'Regional'

# Nacional: solo si tiene PN y NO tiene PR
df_unificado.loc[
    (df_unificado['municipio_receptor_PN'].notna()) & (df_unificado['municipio_receptor_PR'].isna()), 
    'Tipo_Portabilidad'
] = 'Nacional'

# Mostrar detalles de validación
print(f"\n=== VALIDACIÓN DE ASIGNACIÓN ===")
print(f"\nConteos de Tipo_Portabilidad:")
print(df_unificado['Tipo_Portabilidad'].value_counts())

solo_PN = (df_unificado['municipio_receptor_PN'].notna()) & (df_unificado['municipio_receptor_PR'].isna())
solo_PR = (df_unificado['municipio_receptor_PR'].notna()) & (df_unificado['municipio_receptor_PN'].isna())

print(f"\nDesglose detallado:")
print(f"  Solo PN (Nacional): {solo_PN.sum()}")
print(f"  Solo PR (Regional): {solo_PR.sum()}")
print(f"  PN y PR (Regional sobrescribe): {cantidad_ambas}")
print(f"  Sin portabilidad: {(df_unificado['Tipo_Portabilidad'] == 'Sin portabilidad').sum()}")

# Eliminar columnas intermedias
df_unificado = df_unificado.drop(columns=['municipio_receptor_PN', 'departamento_receptor_PN', 'municipio_receptor_PR', 'departamento_receptor_PR'])

print(f"\n=== RESULTADO FINAL ===")
print(f"Ejemplos de registros:")
print(df_unificado[['Tp_Dc', 'Num_coti', 'municipio_receptor', 'departamento_receptor', 'Tipo_Portabilidad']].head(15))

In [ ]:
df_municipios_sie
df_unificado

In [ ]:
# Crear diccionarios de búsqueda desde df_municipios_sie
# Mapeo: (nombre_municipio, nombre_departamento) -> código_municipio
municipios_lookup = {}
for idx, row in df_municipios_sie.iterrows():
    # Verificar que no sean valores nulos
    if pd.notna(row['descripcion']) and pd.notna(row['descripcion_departamento']):
        key = (str(row['descripcion']).strip(), str(row['descripcion_departamento']).strip())
        municipios_lookup[key] = row['municipio']

# Crear diccionario para nombres de municipio -> código
municipio_nombre_codigo = {}
for idx, row in df_municipios_sie.iterrows():
    if pd.notna(row['descripcion']):
        municipio_nombre_codigo[str(row['descripcion']).strip()] = row['municipio']

print(f"Municipios en lookup: {len(municipios_lookup)}")
print(f"Municipios en nombre_codigo: {len(municipio_nombre_codigo)}")

# Función para asignar código de red
def asignar_codigo_red(row):
    # Si tiene municipio_receptor (tiene portabilidad)
    if pd.notna(row['municipio_receptor']) and pd.notna(row['departamento_receptor']):
        # Buscar el código en df_municipios_sie usando municipio_receptor y departamento_receptor
        key = (str(row['municipio_receptor']).strip(), str(row['departamento_receptor']).strip())
        codigo = municipios_lookup.get(key)
        if codigo:
            return codigo
        # Si no encuentra, intentar solo con el municipio_receptor
        return municipio_nombre_codigo.get(str(row['municipio_receptor']).strip())
    # Si no tiene portabilidad, usar Departamento + Municipio
    return row['Departamento'] + row['Municipio']

# Aplicar la función para obtener código de red
df_unificado['municipio_codigo_red'] = df_unificado.apply(asignar_codigo_red, axis=1)

# Función para asignar nombre del municipio donde se asigna la red
def asignar_municipio_nombre(row):
    # Si tiene municipio_receptor, usar ese
    if pd.notna(row['municipio_receptor']):
        return str(row['municipio_receptor']).strip()
    # Si no, buscar en el lookup usando el código combinado
    codigo = row['Departamento'] + row['Municipio']
    # Buscar en df_municipios_sie por código
    resultado = df_municipios_sie[df_municipios_sie['municipio'] == codigo]
    if not resultado.empty:
        desc = resultado.iloc[0]['descripcion']
        if pd.notna(desc):
            return str(desc).strip()
    return None

# Aplicar función para obtener nombre del municipio
df_unificado['municipio_nombre_red'] = df_unificado.apply(asignar_municipio_nombre, axis=1)

# Mostrar resultado
print(f"\nRegistros con municipio_código_red asignado: {df_unificado['municipio_codigo_red'].notna().sum()}")
print(f"Registros con municipio_nombre_red asignado: {df_unificado['municipio_nombre_red'].notna().sum()}")
print(f"Registros con ambos vacíos: {(df_unificado['municipio_codigo_red'].isna() & df_unificado['municipio_nombre_red'].isna()).sum()}")

print(f"\nEjemplos CON Portabilidad:")
con_portabilidad = df_unificado[df_unificado['Tipo_Portabilidad'] != 'Sin portabilidad'].head(5)
print(con_portabilidad[['Departamento', 'Municipio', 'municipio_receptor', 'departamento_receptor', 'municipio_codigo_red', 'municipio_nombre_red', 'Tipo_Portabilidad']])

print(f"\nEjemplos SIN Portabilidad:")
sin_portabilidad = df_unificado[df_unificado['Tipo_Portabilidad'] == 'Sin portabilidad'].head(5)
print(sin_portabilidad[['Departamento', 'Municipio', 'municipio_receptor', 'departamento_receptor', 'municipio_codigo_red', 'municipio_nombre_red', 'Tipo_Portabilidad']])

# Guardar dataframe

In [ ]:
# Crear dataframe de logs con información de los archivos procesados
logs_data = {
    'Archivo': [
        'COMPENSADOS',
        'LMA',
        'Maestro ADRES EPS025',
        'Maestro ADRES EPSC25',
        'Portabilidad Nacional EPSC25',
        'Portabilidad Nacional EPS025',
        'Portabilidad Regional EPSC25',
        'Portabilidad Regional EPS025'
    ],
    'Ruta': [
        R_Compensados,
        r_LMA,
        r_MsEPS025,
        r_MsEPSC25,
        r_P_Nacional_EPSC25,
        r_P_Nacional_EPS025,
        r_P_Regional_EPSC25,
        r_P_Regional_EPS025
    ],
    'Fecha_Proceso': [datetime.now()] * 8
}

df_logs = pd.DataFrame(logs_data)

# Guardar df_unificado en Excel con múltiples hojas
output_path = Raiz + r"\df_unificado_final.xlsx"
with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    df_unificado.to_excel(writer, index=False, sheet_name='Unificado')
    df_logs.to_excel(writer, index=False, sheet_name='Logs')

print(f"Archivo guardado en: {output_path}")
print(f"Total de registros guardados: {len(df_unificado)}")
print(f"\nArchivos procesados:")
print(df_logs)